In [ ]:
!pip install transformers
!pip install sentencepiece

Upload model(model.zip)

In [ ]:
from google.colab import files
uploaded = files.upload()

Unzip

In [ ]:
!unzip model.zip

In [ ]:

from transformers import BertTokenizerFast, BertForSequenceClassification
import torch
device='cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('./model',problem_type="multi_label_classification")
model.to(device)

Prepare predict function

In [6]:
model.eval()
import torch.nn.functional as F
def predict(content,threshold=.5):

    inputs = tokenizer(content,
                       #return_offsets_mapping=True,
                       padding='max_length',
                       truncation=True, return_tensors="pt")

    # move to gpu
    ids = inputs["input_ids"].to(device)
    idt = inputs["token_type_ids"].to(device)
    #print(inputs["input_ids"])
    mask = inputs["attention_mask"].to(device)
    # forward pass
    outputs = model(ids,token_type_ids=idt,attention_mask=mask)
    logits = outputs[0]
    
    
    
    # x = F.sigmoid(logits)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(logits))
    print(probs)
    # active_logits = logits.view(-1, model.num_labels)  # shape (batch_size * seq_len, num_labels)
    
    # print(active_logits.cpu().detach().numpy()[0])
    # flattened_predictions = active_logits.cpu().detach().numpy()[0]
    flattened_predictions = probs.cpu().detach().numpy()[0]
    label = []
    
    for i in range(len(flattened_predictions)):
      if flattened_predictions[i] >= threshold:
        label.append(i)

    return flattened_predictions[0],[3] if label == [] else label

In [ ]:
#upload test data, still tr.csv
from google.colab import files
uploaded = files.upload()

Prepare test data

In [9]:
import pandas as pd
df = pd.read_csv("tr.csv")

In [10]:
sentence = df.irsen_text.values.tolist()[7000:]
# real = df.one_hot_labels.values.tolist()[7000:]
real_c = df.claim_s.values.tolist()[7000:]
real_e = df.evid_s.values.tolist()[7000:]

Predict the test set

In [ ]:
pre_c = []
pre_e = []
pro_c = []# claim probability
for i in range(len(sentence)):
  y, x = predict(sentence[i])
  pro_c.append(y)
  if 0 in x and 1 in x:
    pre_c.append(1)
    pre_e.append(1)
  elif 1 in x:
    pre_c.append(0)
    pre_e.append(1)
  elif 0 in x:
    pre_c.append(1)
    pre_e.append(0)
  else:
    pre_c.append(0)
    pre_e.append(0)

Show result

In [ ]:
print('f1:'+str(f1_score(real_c, pre_c, average=None))+'\n'+'recall:'+str(recall_score(real_c, pre_c, average=None))+'\n'+'precision:'+str(precision_score(real_c, pre_c, average=None))+'\n'+'accuracy:'+str(accuracy_score(real_c, pre_c))+'\n')

In [ ]:
confusion_matrix(real_c, pre_c)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
fig, ax = plt.subplots(1,1,figsize = (10,5))
ax.hist(np.array(pro_c)[np.array(real_c) == 1], color = "darkred",bins = "scott", alpha = .5, edgecolor = "red")
ax.hist(np.array(pro_c)[np.array(real_c) == 0], color = "darkgreen",bins = "scott", alpha = .5, edgecolor = "green")

In [ ]:
# print the wrong predictions. The end of sentence shows the real labels
for i in range(len(real_c)):
  if real_c[i] != pre_c[i]:
    print(sentence[i] + ' claim' if real_c[i] else sentence[i] + ' noclaim')